# Introduction

    - This classifier is trained on a dataset which has images of humans and horses.
    - Key points about the classifier are as follows:
        - Uses Tensorflow & Keras
        - Uses ImageGenerator to read images and auto label them
        - Uses a fancy CNN architecture (sorry fancy because it has multiple CNN layers)
        
    - Nothing more fancy beyond this really :)

In [1]:
# Run this on unix machine to download the zip file
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip

--2019-07-09 17:23:15--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   135MB/s    in 1.1s    

2019-07-09 17:23:16 (135 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]



In [13]:
!unzip -qq horse-or-human.zip -d /tmp/horse-or-human

replace /tmp/horse-or-human/horses/horse01-0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [0]:
import tensorflow as tf
import numpy as np
import os

In [15]:
# Create training ImageGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1/255)

train_gen = train_generator.flow_from_directory(directory='/tmp/horse-or-human',
                                    target_size=(300,300), 
                                    class_mode='binary', 
                                    batch_size=30)

Found 1027 images belonging to 2 classes.


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# define model
model = Sequential()
model.add(Conv2D(filters=100, kernel_size=(3,3), input_shape=(300,300,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=100, kernel_size=(3,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=100, kernel_size=(3,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit_generator(train_gen,                                                    
                    steps_per_epoch=10, 
                    epochs=20)

Epoch 1/20
10/10 [==============================] - 3s 300ms/step - loss: 7.8515 - acc: 0.4633
Epoch 2/20
10/10 [==============================] - 2s 181ms/step - loss: 1.8749 - acc: 0.6000
Epoch 3/20
10/10 [==============================] - 3s 261ms/step - loss: 1.3420 - acc: 0.6390
Epoch 4/20
10/10 [==============================] - 2s 200ms/step - loss: 0.5799 - acc: 0.7767
Epoch 5/20
10/10 [==============================] - 2s 217ms/step - loss: 0.4604 - acc: 0.8333
Epoch 6/20
10/10 [==============================] - 2s 200ms/step - loss: 0.9528 - acc: 0.8087
Epoch 7/20
10/10 [==============================] - 2s 216ms/step - loss: 0.3048 - acc: 0.8900
Epoch 8/20
10/10 [==============================] - 2s 202ms/step - loss: 0.2149 - acc: 0.9206
Epoch 9/20
10/10 [==============================] - 2s 216ms/step - loss: 0.2173 - acc: 0.9267
Epoch 10/20
10/10 [==============================] - 2s 216ms/step - loss: 0.0970 - acc: 0.9700
Epoch 11/20
10/10 [==============================

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 298, 298, 100)     2800      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 149, 149, 100)     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 147, 147, 100)     90100     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 100)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 71, 71, 100)       90100     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 100)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 122500)           

In [43]:
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()


Saving woman3.jpg to woman3.jpg
Saving woman2.jpg to woman2.jpg
Saving woman1.jpg to woman1.jpg
Saving men3.jpg to men3.jpg
Saving men2.jpg to men2.jpg
Saving men1.jpg to men1.jpg


In [44]:
for uploaded_keys in uploaded.keys():
  path = '/content/'+uploaded_keys
  uploaded_image = image.load_img(path, target_size=(300,300))
  
  uploaded_image_array = image.img_to_array(uploaded_image)
  uploaded_image_array = uploaded_image_array.reshape(1, uploaded_image_array.shape[0],uploaded_image_array.shape[1],uploaded_image_array.shape[2])
  
  prediction = model.predict(uploaded_image_array)
  if (prediction>0.5):
    print ('Image is of human')
  else :
    print ('Image is of horse')

Image is of horse
Image is of human
Image is of human
Image is of human
Image is of horse
Image is of human


## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)